I need a small testing playground that won't crash on my laptop, so I'm creating one here for the 10000 frame centriole data.

In [1]:
%pylab
import DataSTORM.processors as ds
import pandas as pd
from pathlib import Path

Using matplotlib backend: Qt4Agg
Populating the interactive namespace from numpy and matplotlib


In [2]:
filename = Path('../test-data/acTub_COT_gain100_30ms/acTub_COT_gain100_30ms.csv')
with open(str(filename.resolve()), 'r') as file:
    df = pd.read_csv(file, engine = 'c')

In [3]:
df.describe()

,x,x-uncertainty,y,y-uncertainty,frame,ad-count,chi-square,background
count,214717.000000,214717.000000,214717.000000,214717.000000,214717.000000,214717.000000,214717.000000,214717.000000
mean,2425.196663,35.420145,2182.946881,35.420145,5172.075346,6288.638717,1029.127987,744.422318
std,1595.345619,20.475942,1322.208205,20.475942,2777.221724,15586.487514,2896.254531,312.086668
min,0.000001,0.847267,0.000002,0.847267,0.000000,338.324000,50.035500,523.371000
25%,965.218000,20.430100,982.750000,20.430100,2904.000000,577.636000,222.657000,613.929000
50%,2403.570000,38.192700,2206.350000,38.192700,5165.000000,772.204000,304.708000,656.426000
75%,3892.590000,50.197000,3461.780000,50.197000,7563.000000,1596.560000,433.924000,730.572000
max,4900.000000,80.431400,4200.000000,80.431400,9999.000000,177421.000000,58298.300000,7147.530000


In [4]:
import importlib

In [80]:
maxSize = np.max([df['x'].max(),df['y'].max()])
pixelSize = 100 # nm

fig = plt.figure()
ax  = fig.add_subplot(111)
ax.hist2d(df['x'], df['y'], bins = maxSize / 100)
ax.invert_yaxis()
plt.show()

In [188]:
plt.close()

In [71]:
importlib.reload(ds)

<module 'DataSTORM.processors' from '/home/kmdouglass/src/DataSTORM/DataSTORM/processors.py'>

In [72]:
corr = ds.FiducialDriftCorrect(mergeRadius      = 90,
                               offTime          = 3,
                               minSegmentLength = 30,
                               searchRegions    = [{'xMin' : 1500,
                                                    'xMax' : 1700,
                                                    'yMin' : 2500,
                                                    'yMax' : 2700}])

In [73]:
corr.iSearch(df)

Space bar pressed!
Closed Figure!


/home/kmdouglass/anaconda3/envs/DataSTORM/lib/python3.5/site-packages/matplotlib/backend_bases.py:2435: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


In [74]:
plt.close()

In [75]:
df2 = corr(df)

Frame 9996: 1 trajectories present
1 fiducial(s) detected.


In [76]:
df2.describe()

,ad-count,background,chi-square,frame,x,x-uncertainty,y,y-uncertainty,dx,dy
count,205975.000000,205975.000000,205975.000000,205975.000000,205975.000000,205975.000000,205975.000000,205975.000000,205975.000000,205975.000000
mean,5961.789793,742.002304,1000.376989,5172.442277,2449.917511,36.751226,2191.310031,36.751226,11.136573,-23.985763
std,15735.554133,316.370985,2900.234056,2775.162190,1618.858941,19.834358,1348.258207,19.834358,6.764342,21.478217
min,338.324000,523.371000,50.035500,0.000000,-21.343824,0.847267,-4.655255,0.847267,-10.558332,-58.577148
25%,570.513000,611.951500,219.725000,2909.000000,889.910959,24.931950,963.737642,24.931950,10.640409,-44.893025
50%,752.848000,652.932000,297.752000,5167.000000,2519.141423,39.065900,2104.718976,39.065900,13.371441,-23.280464
75%,1255.420000,723.177500,399.925000,7557.000000,3945.439926,50.801700,3515.060045,50.801700,14.771622,-1.980576
max,177421.000000,7147.530000,58298.300000,9999.000000,4910.558288,80.431400,4258.577148,80.431400,21.348286,4.657005


In [81]:
maxSize = np.max([df2['x'].max(),df2['y'].max()])
pixelSize = 100 # nm

fig = plt.figure()
ax  = fig.add_subplot(111)
ax.hist2d(df2['x'], df2['y'], bins = maxSize / 100)
ax.invert_yaxis()
plt.show()

In [228]:
corr.searchRegions

[{'xMax': 1630.0403218824745,
  'xMin': 1469.5060474140912,
  'yMax': 2668.7500006947203,
  'yMin': 2450.0000007939661}]

In [186]:
corr.resetSearchRegions()
corr.searchRegions

[{'xMax': None, 'xMin': None, 'yMax': None, 'yMin': None}]

In [77]:
pd.concat([df, corr.fiducialTrajectories[0]]).drop_duplicates(subset = ['x', 'y'], keep = False).describe()

,ad-count,background,chi-square,frame,particle,x,x-uncertainty,y,y-uncertainty
count,205975.000000,205975.000000,205975.000000,205975.000000,0,205975.000000,205975.000000,205975.000000,205975.000000
mean,5961.789793,742.002304,1000.376989,5172.442277,NaN,2461.054084,36.751226,2167.324268,36.751226
std,15735.554133,316.370985,2900.234056,2775.162190,NaN,1618.947414,19.834358,1347.698495,19.834358
min,338.324000,523.371000,50.035500,0.000000,NaN,0.000001,0.847267,0.000002,0.847267
25%,570.513000,611.951500,219.725000,2909.000000,NaN,900.397000,24.931950,940.650000,24.931950
50%,752.848000,652.932000,297.752000,5167.000000,NaN,2530.090000,39.065900,2080.190000,39.065900
75%,1255.420000,723.177500,399.925000,7557.000000,NaN,3957.640000,50.801700,3492.330000,50.801700
max,177421.000000,7147.530000,58298.300000,9999.000000,NaN,4900.000000,80.431400,4200.000000,80.431400


In [64]:
df2.drop_duplicates(subset=['x','y'], keep = False).describe()

,ad-count,background,chi-square,frame,particle,x,x-uncertainty,y,y-uncertainty,dx,dy
count,206018.000000,206018.000000,206018.00000,206018.000000,0,206018.000000,206018.000000,206018.000000,206018.000000,206018.000000,206018.000000
mean,5963.846659,742.039032,1000.65586,5172.364405,NaN,2450.246146,36.746808,2191.500105,36.746808,10.773170,-24.068002
std,15738.685082,316.365634,2900.35229,2775.325997,NaN,1618.832996,19.836548,1348.180410,19.836548,6.794132,21.476977
min,338.324000,523.371000,50.03550,0.000000,NaN,-21.016902,0.847267,-4.604117,0.847267,-11.277247,-58.710463
25%,570.535250,611.960250,219.73225,2909.000000,NaN,890.537065,24.921125,963.986169,24.921125,10.285282,-44.950636
50%,752.917500,652.947000,297.76150,5167.000000,NaN,2518.918281,39.063350,2105.410782,39.063350,13.024147,-23.392957
75%,1255.942500,723.246000,399.98675,7557.000000,NaN,3945.801184,50.799400,3515.008229,50.799400,14.341150,-2.082358
max,177421.000000,7147.530000,58298.30000,9999.000000,NaN,4911.277206,80.431400,4258.710463,80.431400,21.021277,4.605352
